In [1]:
import numpy as np
import matplotlib.colors as colors
import matplotlib.pyplot as plt
# import seaborn as sns
import pandas as pd

import sklearn
import scipy

import sys
import os
import pathlib
import itertools
import glob
import re

package_path = os.path.join(pathlib.Path.home(), "mkvchain")
sys.path.append(package_path)
from model import FeatureDependentMarkovChain

In [3]:
# root = pathlib.Path.home()
# pt = os.path.join(root, 'research', 'GADS_Processed', "trajectories_with_features")

# def glob_re(pattern, strings):
#     return list(filter(re.compile(pattern).match, strings))

# filenames = glob_re(r"gen_\d+_type_200_dtgrp_\d+_rating_\d+_state_Texas_raw.csv", os.listdir(pt))

# mylist = []

# for f in filenames:
#     data = pd.read_csv(os.path.join(pt,f))
#     data.set_index(pd.DatetimeIndex(data["x"]), inplace=True)
#     data = data[~data.index.duplicated()]
#     data.rename(columns={"COAST": "y5", "NORTH_C" : "y5"}, inplace=True)
#     mylist.append(data)


In [2]:
root = pathlib.Path.home()
pt1 = os.path.join(root, 'research', 'GADS_Processed', "trajectories_with_features")
pt2 = os.path.join(root, 'research', 'ERCOT_WEATHER_DATA', "processed", "texas_coast_weather_daily_2013_2023.csv.csv")
weather = pd.read_csv(pt2)
weather.set_index(pd.DatetimeIndex(weather["DATE"]), inplace=True)

def glob_re(pattern, strings):
    return list(filter(re.compile(pattern).match, strings))

filenames = glob_re(r"gen_\d+_type_200_dtgrp_\d+_rating_\d+_state_Texas_raw.csv", os.listdir(pt1))

mylist = []

for f in filenames:
    data = pd.read_csv(os.path.join(pt1,f))
    data.set_index(pd.DatetimeIndex(data["x"]), inplace=True)
    data = data[~data.index.duplicated()]
    # data.rename(columns={"COAST": "y5", "NORTH_C" : "y5"}, inplace=True)
    data = data.join(weather)
    mylist.append(data)


In [3]:
for d in mylist:  
    # po_idx = np.where(np.logical_or(d["y2"].values == 3, d["y2"].values == 4))[0]
    po_idx = np.where(d["y2"].values == 3)[0]
    end_of_event = np.concat([[False], np.diff(po_idx) != 1])
    end_of_event_idx = po_idx[end_of_event]

    idx = range(len(d))
    my_data = np.zeros(len(d))

    for i in idx:
        events_before_i = np.array([j for j in end_of_event_idx if j < i])
        if len(events_before_i) > 1:
            closest_idx = np.argmin(i - events_before_i)
            my_event_idx = events_before_i[closest_idx]
            hours_since_event = (i - my_event_idx)
        else:
            hours_since_event = None
        my_data[i] = hours_since_event
    
    d["y6"] = my_data

    # mo_idx = np.where(d["y2"].values == 4)[0]
    # if len(mo_idx) < 1:
    #     my_data = None
    # else:
    #     end_of_event = np.concat([[False], np.diff(mo_idx) != 1])
    #     end_of_event_idx = mo_idx[end_of_event]

    #     idx = range(len(d))
    #     my_data = np.zeros(len(d))

    #     for i in idx:
    #         events_before_i = np.array([j for j in end_of_event_idx if j < i])
    #         if len(events_before_i) > 1:
    #             closest_idx = np.argmin(i - events_before_i)
    #             my_event_idx = events_before_i[closest_idx]
    #             hours_since_event = (i - my_event_idx)
    #         else:
    #             hours_since_event = None
    #         my_data[i] = hours_since_event
    
    # d["y7"] = my_data

    d["y8"] = d.index.month

In [ ]:

# features = d.loc[(~np.isnan(d["y6"])) & (~np.isnan(d["y5"])), ["y3", "y5", "y4", "y6"]]
# states = d.loc[features.index, "y2"].values

# print(sklearn.feature_selection.r_regression(features, states))
# print(sklearn.feature_selection.chi2(features, states))


In [6]:
d = mylist[0]
for c in ['y3', 'COAST', 'y4', 'PRCP', 'TMAX', 'TMIN', 'y6', 'y7', 'y8']:
    contingency_table = pd.crosstab(d["y2"], d[c])
    try:
        res = scipy.stats.chi2_contingency(contingency_table)
    except:
        continue
    print(c, res.pvalue)

y3 0.0
COAST 0.00015329933491778217
y4 0.0
PRCP 0.0
TMAX 0.0
TMIN 0.0
y6 0.03197595686923424
y8 0.0


In [7]:
d = mylist[1]
for c in ['y3', 'COAST', 'y4', 'PRCP', 'TMAX', 'TMIN', 'y6', 'y7', 'y8']:
    contingency_table = pd.crosstab(d["y2"], d[c])
    try:
        res = scipy.stats.chi2_contingency(contingency_table)
    except:
        continue
    print(c, res.pvalue)

y3 0.0
COAST 0.010746434739238293
y4 0.0
PRCP 0.0
TMAX 0.0
TMIN 0.0
y6 0.0
y8 0.0


In [12]:
d['y7']

x
2013-01-01 06:00:00    None
2013-01-01 07:00:00    None
2013-01-01 08:00:00    None
2013-01-01 09:00:00    None
2013-01-01 10:00:00    None
                       ... 
2023-12-31 20:00:00    None
2023-12-31 21:00:00    None
2023-12-31 22:00:00    None
2023-12-31 23:00:00    None
2024-01-01 00:00:00    None
Name: y7, Length: 96392, dtype: object

In [4]:
states_new = []
features_new = []
lengths_new = []

for d in mylist:
    states = d["y2"].values.tolist()
    features = d[['y3', 'COAST', 'y4', 'PRCP', 'TMAX', 'TMIN', 'y6', 'y8']].values
    l = len(d)

    states_new += [states]
    features_new += [features]
    lengths_new += [l]

states = np.concatenate(states_new).astype(int)
states -= 1
features = np.vstack(features_new)
lengths = np.array(lengths_new)

In [26]:
np.unique(states)

array([0, 1, 2, 3])

In [5]:
train_idx = 1 # int(lengths.size * .8)
val_idx = 2 #int(lengths.size * .9)

lengths_train = lengths[:train_idx]
lengths_val = lengths[train_idx:val_idx]
# lengths_test = lengths[val_idx:]

states_train = states[:lengths_train.sum()]
states_val = states[lengths_train.sum():lengths_train.sum()+lengths_val.sum()]
# states_test = states[lengths_train.sum()+lengths_val.sum():]

features_train = features[:lengths_train.sum()]
features_val = features[lengths_train.sum():lengths_train.sum()+lengths_val.sum()]
# features_test = features[lengths_train.sum()+lengths_val.sum():]

In [6]:
from sklearn.preprocessing import QuantileTransformer, StandardScaler
ss = StandardScaler()

features_train = ss.fit_transform(features_train)
features_val = ss.transform(features_val)
# features_test = ss.transform(features_test)

In [19]:
features_train

array([[-1.73227784, -1.79290221, -1.72762601, ..., -0.33909146,
                nan, -1.6020725 ],
       [-1.73227784, -1.74857076, -1.72762601, ..., -0.33909146,
                nan, -1.6020725 ],
       [-1.73227784, -1.73272132, -1.72762601, ..., -0.33909146,
                nan, -1.6020725 ],
       ...,
       [ 1.73200392, -0.28931338,  1.72547345, ..., -1.97503395,
        -0.71249848,  1.58788438],
       [ 1.73200392, -0.38119227,  1.72547345, ..., -1.97503395,
        -0.71199974,  1.58788438],
       [ 1.73200392, -0.47795592, -1.72762601, ..., -1.54826635,
        -0.711501  , -1.6020725 ]], shape=(96392, 8))

In [7]:
n = 4
model1 = FeatureDependentMarkovChain(n, lam_frob=0, n_iter=1)
model1.fit(states_train, features_train*0, lengths_train, verbose=False)
train1, val1 = model1.score(states_train, features_train*0, lengths_train, average=False), \
    model1.score(states_val, features_val*0, lengths_val, average=False)
    # model1.score(states_test, features_test*0, lengths_test, average=False)
train1, val1

(np.float64(-773.996324337362), np.float64(-593.8767097560176))

In [8]:
predictions = model1.predict(features_val)
for i, j in itertools.product(range(4), range(4)):
    y = predictions[:, i, j]
    val = len(np.unique(y))
    print(f"{i} -> {j} : {val}")

0 -> 0 : 2
0 -> 1 : 2
0 -> 2 : 2
0 -> 3 : 2
1 -> 0 : 2
1 -> 1 : 2
1 -> 2 : 2
1 -> 3 : 2
2 -> 0 : 2
2 -> 1 : 2
2 -> 2 : 2
2 -> 3 : 2
3 -> 0 : 2
3 -> 1 : 2
3 -> 2 : 2
3 -> 3 : 2


In [9]:
from copy import deepcopy
train2, val2, test2 = -np.inf, -np.inf, -np.inf
best_lam = None
model2 = None
for lam in np.logspace(-3,-1,10):
    model = FeatureDependentMarkovChain(n, lam_frob=lam,  n_iter=1)
    model.As = deepcopy(model1.As)
    model.bs = deepcopy(model1.bs)
    model.fit(states_train, features_train, lengths_train, verbose=False, warm_start=True)
    traini, vali= model.score(states_train, features_train, lengths_train, average=False), \
          model.score(states_val, features_val, lengths_val, average=False)
        #   model.score(states_test, features_test, lengths_test, average=False)
    if vali > val2:
        train2 = traini
        val2 = vali
        # test2 = testi
        best_lam = lam
        model2 = model
train2, val2# test2

(np.float64(-762.7624227693096), np.float64(-584.0683045673397))

In [10]:
model = FeatureDependentMarkovChain(n, lam_frob=best_lam, n_iter=1)
model.As = deepcopy(model1.As)
model.bs = deepcopy(model1.bs)
model.fit(states_train, features_train, lengths_train, verbose=False, warm_start=True)

In [11]:
predictions2 = model.predict(features_val)
for i, j in itertools.product(range(4), range(4)):
    y = predictions2[:, i, j]
    val = len(np.unique(y.round(decimals=4)))
    print(f"{i} -> {j} : {val}")

0 -> 0 : 9
0 -> 1 : 4
0 -> 2 : 6
0 -> 3 : 2
1 -> 0 : 255
1 -> 1 : 263
1 -> 2 : 7
1 -> 3 : 2
2 -> 0 : 41
2 -> 1 : 2
2 -> 2 : 41
2 -> 3 : 2
3 -> 0 : 11
3 -> 1 : 2
3 -> 2 : 2
3 -> 3 : 11


In [ ]:
j = np.where(model.nonzero[0] == 0)[0][0]
A, b = model.As[0], model.bs[0]
feature_names = np.array(['y3', 'COAST', 'y4', 'PRCP', 'TMAX', 'TMIN', 'y6', 'y8'])
print(feature_names[np.argsort(A[:,j])[-5:]]) # month, TMAX', 'PRCP', age, load
print(feature_names[np.argsort(A[:,j])[:5]]) # time since last planned outage, doy, month

['y8' 'TMAX' 'PRCP' 'y3' 'COAST']
['y6' 'TMIN' 'y4' 'y8' 'TMAX']


In [16]:
np.where(model.nonzero[0] == 0)[0][0]

np.int64(0)